# OMOP CDM Query Experiments with PyOMOP and LLMs

This notebook demonstrates how to use the PyOMOP library and Large Language Models (LLMs) to query an OMOP Common Data Model (CDM) database. 

**Instructions:**
- Configure your LLM API keys as environment variables if needed. (Supports all LLMs compatible with LangChain)
- Run each cell in order and review the comments for guidance

---

In [ ]:
!pip install pyomop[llm]
!rm cdm.sqlite
!pyomop -e Synthea27Nj -v 5.4 -n cdm.sqlite

In [9]:
import asyncio
import os
import re
from dotenv import load_dotenv

load_dotenv()
# Import langchain LLM components
from langchain_google_genai import ChatGoogleGenerativeAI
from dhti_elixir_base import BaseLLM, BaseChatLLM
from sqlalchemy import text
from langchain.chat_models import init_chat_model

from pyomop import CDMDatabase, CdmEngineFactory, CdmLLMQuery, CdmVector

## You may use any LLM supported by langchain.

### Alternate LLM Examples

```python
    # Alternative LLM examples:
    llm = ChatOpenAI(model="gpt-4", api_key=os.getenv("OPENAI_API_KEY")
    llm = ChatAnthropic(model="claude-3-opus-20240229", api_key=os.getenv("ANTHROPIC_API_KEY")
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", google_api_key=os.getenv("GOOGLE_API_KEY")

In [15]:
llm = init_chat_model(
    model="mistralai/devstral-2512:free",
    model_provider="openai",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

In [16]:
cdm = CdmEngineFactory(
    db="sqlite",
    name="cdm.sqlite",
)
engine = cdm.engine
# Define the important OMOP CDM tables to include in the query context
# These are the most commonly used tables for clinical research queries
important_tables = [
    "person",  # Patient demographics
    "observation_period",  # Patient observation periods
    "visit_occurrence",  # Healthcare visits
    "condition_era", # Condition era table
    "condition_occurrence",  # Diagnoses
    "drug_era", # Drug era table
    "drug_exposure",  # Medications
    "procedure_occurrence",  # Procedures
    "measurement",  # Lab results and vitals
    "observation_period",  # Observation periods
    "observation",  # General clinical observations
    "death",  # Mortality
    "concept",  # Vocabularies (for lookups)
    "provider",  # Healthcare providers
]

# Create SQL database wrapper with OMOP CDM metadata
sql_database = CDMDatabase(
    engine,  # type: ignore
    include_tables=important_tables,
    version="cdm54",  # Use 'cdm6' for CDM version 6.0
)

# Create LLM-powered query engine
query_engine = CdmLLMQuery(
    sql_database,
    llm=llm,
).query_engine

In [17]:
query = "What are the top 5 most common conditions?"

In [18]:
try:
    # Execute query using LLM
    print("🤖 Querying with LLM...")
    response = await query_engine.ainvoke({"input": query})
    print("✅ Response received:")
except Exception as e:
    print(f"❌ Error: {e}")
    print()

🤖 Querying with LLM...


> Entering new SQL Agent Executor chain...
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.  Then I should query the database for the top 5 most common conditions.

concept, condition_era, condition_occurrence, death, drug_era, drug_exposure, measurement, observation, observation_period, person, procedure_occurrence, provider, visit_occurrenceINFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"

Invoking: `sql_db_schema` with `{'table_names': 'condition_occurrence'}`
responded: Now I need to look at the schema of the condition_occurrence table to see what columns are available:


CREATE TABLE condition_occurrence (
	condition_occurrence_id BIGINT, 
	person_id BIGINT, 
	condition_co

In [19]:

print(f"LLM Response: {response['output']}")
print()

LLM Response: The top 5 most common conditions are:

1. Stress
2. Viral sinusitis
3. Social isolation
4. Limited social contact
5. Acute viral pharyngitis

